In [1]:
from utils.all import *
data = load_data('../annotated', batch_num=[5, 6, 7, 8, 9, 10, 11], preprocess=False)

for sent in data:
    sent['system'] = sent['system'] \
        .replace('new-wiki-2', 'simpeval-22') \
        .replace('new-wiki-1', 'simpeval-22') \
        .replace(' Writing', '-written') \
        .replace('Human ', 'Human-') \
        .replace('new-wiki-3', 'simpeval-ext')

simpeval_22, simpeval_ext = [], []
for sent in data:
    if 'simpeval-22' in sent['system']:
        simpeval_22 += [sent]
    elif 'simpeval-ext' in sent['system']:
        simpeval_ext += [sent]

Found users: {'vishnesh', 'rachel', 'kelly', 'anton', 'ayush', 'vinayak'}



In [2]:
def number_annotations_per_system(data):
    systems = set([s['system'] for s in data])
    for system in systems:
        print(f"{len([s for s in data if s['system'] == system])}: {system}")
number_annotations_per_system(simpeval_22)
print('')
number_annotations_per_system(simpeval_ext)

180: simpeval-22/Muss
180: simpeval-22/T5-3B
180: simpeval-22/Human-1-written
180: simpeval-22/GPT-3-zero-shot
180: simpeval-22/Human-2-written
180: simpeval-22/T5-11B
180: simpeval-22/GPT-3-few-shot

120: simpeval-ext/Human-1-written
120: simpeval-ext/GPT-3-zero-shot
120: simpeval-ext/T5-11B
120: simpeval-ext/GPT-3-few-shot
120: simpeval-ext/Human-2-written
120: simpeval-ext/Muss
120: simpeval-ext/T5-3B


In [3]:
with open(f"../salsa/simpeval_22.json", "w") as f:
   json.dump(simpeval_22, f, indent=4)

with open(f"../salsa/simpeval_ext.json", "w") as f:
   json.dump(simpeval_ext, f, indent=4)

In [4]:
data = load_data('../annotated', batch_num=[1, 2, 3, 4], preprocess=False)

for sent in data:
    sent['system'] = sent['system'] \
        .replace('new_systems', 'turkcorpus') \
        .replace('new_systems', 'turkcorpus') \
        .replace('systems', 'turkcorpus')

preliminary = data

Found users: {'ayush', 'vishnesh', 'rachel', 'anton', 'kelly', 'vinayak'}



In [5]:
number_annotations_per_system(preliminary)

74: turkcorpus/turk_corpus_random.txt
74: turkcorpus/asset.test.simp.second
74: turkcorpus/T5.txt
74: turkcorpus/asset.test.simp
74: turkcorpus/con_simplification.txt


In [6]:
with open(f"../salsa/turkcorpus_preliminary.json", "w") as f:
   json.dump(preliminary, f, indent=4)